#**antsys**
A general purpose ant colony optimization system.
<br/><br/>**Overview**
<br/>The Ant Colony Optimization (ACO) is a technique, inspired by the foraging behavior of ants, to find good solutions for discrete optimization problems. Its central metaphor resides in the indirect communication mechanism through chemical signals (pheromones) used by many species of social ants in their search for food sources.
<br/>The same inspiration was build in the **antsys** package, wich takes advantage of *python* flexibility to be easily applied to different optimization problems.
<br/><br/>**Installation**
<br/>Installation via ```pip```

In [1]:
!pip3 install antsys

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Usage Example:** *Knapsack Problem*
<br/>The **antsys** package was designed to be easy to apply to different optimization problems. So, the *knapsack problem* was selected as example, since the ACO's application is not straightforward. This problem is based on a situation in which, from a set of objects of different weights and values, you want to fill a *knapsack* obtaining the highest possible value without exceeding its capacity.
<br/>1 - Import necessary packages and modules

In [4]:
from antsys import AntWorld
from antsys import AntSystem
import numpy as np
import random

2 - Generate a knapsack problem instance

In [5]:
# generate and show knapsack capacity
max_weight = random.randint(200,500)
print('knapsack max weight =', max_weight)

# generate and show available items
print('\navailable items:')
print('|item|weight| value|')
items = []
for i in range(20):
  items.append((i, random.randint(50,200), random.randint(100,500)))
  print('|%4i|%6i|%6i|' % items[i])

knapsack max weight = 425

available items:
|item|weight| value|
|   0|    91|   497|
|   1|    68|   391|
|   2|    50|   261|
|   3|    84|   331|
|   4|   127|   125|
|   5|   151|   295|
|   6|    75|   316|
|   7|   184|   338|
|   8|    56|   292|
|   9|    98|   111|
|  10|   169|   160|
|  11|    86|   355|
|  12|   148|   292|
|  13|    99|   452|
|  14|   130|   495|
|  15|   170|   245|
|  16|   124|   419|
|  17|   181|   446|
|  18|    93|   481|
|  19|   109|   301|


3 - The function ```knapsack_rules``` will append information to the edges during the world creation. In this case there will be two edges between pairs of nodes, one assigning (```1```) and the other discharging (```0```) the item (```end```) of being included in the knapsack.

In [6]:
def knapsack_rules(start, end):
  return [0, 1]

4 - The function ```knapsack_cost``` will be used to calculate the cost of any possible solution (```path```).

In [7]:
def knapsack_cost(path):
  k_value = 0
  k_weight = 0
  for edge in path:
    if edge.info == 1:
      k_value += edge.end[2]
      k_weight += edge.end[1]
  cost = 1/k_value
  if k_weight > max_weight:
    cost += 1
  else:
    for edge in path:
      if edge.info == 0 and edge.end[1] <= (max_weight-k_weight):
        cost += 1
  return cost

5 - The ```knapsack_heuristic``` is a simple heuristic that will help the ants to make better choices. The probability to choose an item that fits in the remaining capacity of the knapsack will be slightly higher.


In [8]:
def knapsack_heuristic(path, candidate):
  k_weight = 0
  for edge in path:
    if edge.info == 1:
      k_weight += edge.end[1]
  if candidate.info == 1 and candidate.end[1] < (max_weight-k_weight):
    return 0
  elif candidate.info == 0:
    return 1
  else:
    return 2

6 - This function shows the details of a possible solution (```path```).

In [9]:
def print_solution(path):
  print('knapsack items:')
  print('|item|weight| value|')
  value = 0
  weight = 0
  for edge in path:
    if(edge.info == 1):
      print('|%4i|%6i|%6i|' % edge.end)
      value+=edge.end[2]
      weight+=edge.end[1]
  print('total weight = %g\ntotal value = %g' % (weight, value))

7 - The world (```new_world```) is created from the nodes (```items```) as a directed complete graph. In this point, ```knapsack_rules```, ```knapsack_cost``` and ```knapscack_heuristic``` are defined as respectively ```r_func```, ```c_func``` and ```h_func```. These functions are bound to the world and the first one has an important role in its structure.
* Assigning ```False``` to ```complete```, the world would be as a set of choices to travel across the nodes in sequence. This would work for the present problem, leading to a shorter convergence time and, in most cases, a worse solution. The best choice of world representation depends on the problem.

In [10]:
new_world = AntWorld(items, knapsack_rules, knapsack_cost, knapsack_heuristic, True, 10)

8 - Configure ```ant_opt``` as an ```AntSystem```.

In [11]:
ant_opt = AntSystem(world=new_world, n_ants=100)

9 - Execute the optimization loop.

In [12]:
ant_opt.optimize(100,50)

| iter |         min        |         max        |        best        |
|     1|         0.000509424|             3.00116|         0.000509424|
|     2|         0.000537634|             2.00117|         0.000509424|
|     3|         0.000538503|             2.00079|         0.000509424|
|     4|         0.000515198|             1.00128|         0.000509424|
|     5|         0.000558347|             1.00194|         0.000509424|
|     6|         0.000588582|             1.00108|         0.000509424|
|     7|         0.000513875|             3.00183|         0.000509424|
|     8|         0.000517331|             2.00084|         0.000509424|
|     9|         0.000514403|             2.00124|         0.000509424|
|    10|         0.000510204|             2.00086|         0.000509424|
|    11|         0.000526593|             2.00102|         0.000509424|
|    12|         0.000478927|             2.00102|         0.000478927|
|    13|         0.000511247|             2.00091|         0.000

10 - Show details about the best solution found.

In [13]:
print('\nknapsack max weight =', max_weight)
print_solution(ant_opt.g_best[2])


knapsack max weight = 425
knapsack items:
|item|weight| value|
|  18|    93|   481|
|   0|    91|   497|
|   1|    68|   391|
|  13|    99|   452|
|   8|    56|   292|
total weight = 407
total value = 2113
